In [13]:
from src import loader
from riix.models.elo import Elo
from riix.models.glicko2 import Glicko2
from riix.models.sticko import Sticko
from riix.models.ttt import TrueSkillThroughTime
from riix.utils import MatchupDataset, split_matchup_dataset, generate_matchup_data
from riix.metrics import binary_metrics_suite
#from riix.models.eloplus import EloPlus
from riix.models.glicko import Glicko
import polars as pl
import numpy as np
import trueskill_through_time as ttt
import cProfile
from scipy.optimize import minimize_scalar
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss, brier_score_loss

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
mdb_file = r"/mnt/c/Program Files (x86)/OnCourt/OnCourt.mdb"
password = "qKbE8lWacmYQsZ2"
atp = loader.TennisLoader(mdb_file, password)

In [22]:
atp.clean_games()

In [109]:
dataset = MatchupDataset(
    atp.to_riix_format(sample_games=1000000),
    competitor_cols = ['P1', 'P2'],
    outcome_col = 'Result',
    datetime_col = 'Date',
    rating_period = '1D',
)
train_dataset, test_dataset = split_matchup_dataset(dataset, test_fraction=0.9)

loaded dataset with:
967945 matchups
39262 unique competitors
12839 rating periods of length 1D
split into train_dataset of length 96795 and test_dataset of length 871150


In [124]:
model_Elo = Elo(train_dataset.competitors, k = 90)
model_Elo.fit_dataset(train_dataset)
test_probs_Elo = model_Elo.fit_dataset(test_dataset, return_pre_match_probs=True)
test_metrics_Elo = binary_metrics_suite(probs=test_probs_Elo, outcomes=test_dataset.outcomes)
print(test_metrics_Elo)

{'accuracy': np.float64(0.7043758250588302), 'accuracy_without_draws': np.float64(0.7043758250588302), 'log_loss': np.float64(0.5675423741633863), 'brier_score': np.float64(0.19248680021662)}


In [49]:
model_Glicko = Glicko(train_dataset.competitors, c = 6.33)
model_Glicko.fit_dataset(train_dataset)
test_probs_Glicko = model_Glicko.fit_dataset(test_dataset, return_pre_match_probs=True)
test_metrics_Glicko = binary_metrics_suite(probs=test_probs_Glicko, outcomes=test_dataset.outcomes)
print(test_metrics_Glicko)

{'accuracy': np.float64(0.7089604166666666), 'accuracy_without_draws': np.float64(0.7089604166666666), 'log_loss': np.float64(0.5542591061747458), 'brier_score': np.float64(0.18825878324441994)}


In [50]:
model_Glicko2 = Glicko2(train_dataset.competitors, tau = 9.465461359819516, initial_sigma = 0.04615461031041576)
model_Glicko2.fit_dataset(train_dataset)
test_probs_Glicko2 = model_Glicko2.fit_dataset(test_dataset, return_pre_match_probs=True)
test_metrics_Glicko2 = binary_metrics_suite(probs=test_probs_Glicko2, outcomes=test_dataset.outcomes)
print(test_metrics_Glicko2)

{'accuracy': np.float64(0.7128243055555555), 'accuracy_without_draws': np.float64(0.7128243055555555), 'log_loss': np.float64(0.5500693790495248), 'brier_score': np.float64(0.18635849139655974)}


In [51]:
model_Sticko = Sticko(train_dataset.competitors, c = 7.301939694943711, h = 260.6, beta = 0.023903153784953272)
model_Sticko.fit_dataset(train_dataset)
test_probs_Sticko = model_Sticko.fit_dataset(test_dataset, return_pre_match_probs=True)
test_metrics_Sticko = binary_metrics_suite(probs=test_probs_Sticko, outcomes=test_dataset.outcomes)
print(test_metrics_Sticko)

{'accuracy': np.float64(0.72363125), 'accuracy_without_draws': np.float64(0.72363125), 'log_loss': np.float64(0.5376469009457989), 'brier_score': np.float64(0.18114626818914822)}


In [55]:
n_test = 10000
combined_probs = pl.DataFrame({
    'elo': test_probs_Elo,
    'glicko': test_probs_Glicko,
    'glicko2': test_probs_Glicko2,
    'sticko': test_probs_Sticko,
    'result': test_dataset.outcomes
})
combined_probs_train = combined_probs.head(len(combined_probs) - n_test)
combined_probs_test = combined_probs.tail(n_test)

# Separate features and target for training
X_train = combined_probs_train.select(['elo', 'glicko', 'glicko2', 'sticko'])
y_train = combined_probs_train['result']

# Separate features and target for testing
X_test = combined_probs_test.select(['elo', 'glicko', 'glicko2', 'sticko'])
y_test = combined_probs_test['result']

In [99]:
# Instantiate the XGBoost model
xgb_model = xgb.XGBClassifier(eval_metric='logloss')

# Fit the model using the training data
xgb_model.fit(X_train.to_numpy(), y_train.to_numpy())

# Make predictions on the test data
y_pred = xgb_model.predict(X_test.to_numpy())
y_pred_proba = xgb_model.predict_proba(X_test.to_numpy())

# Evaluate the model's performance
accuracy = accuracy_score(y_test.to_numpy(), y_pred)
logloss = log_loss(y_test.to_numpy(), y_pred_proba)
#brier = brier_score_loss(y_test, y_pred_proba)

print(f"Accuracy: {accuracy}")
print(f"Log Loss: {logloss}")
#print(f"Brier:, {brier}")

Accuracy: 0.7234
Log Loss: 0.5363299514810272


In [96]:
logloss_elo = log_loss(y_test, X_test['elo'].to_numpy())
brier_elo = brier_score_loss(y_test, X_test['elo'].to_numpy())

In [101]:
logloss_sticko = log_loss(y_test, X_test['sticko'].to_numpy())
brier_sticko = brier_score_loss(y_test, X_test['sticko'].to_numpy())

In [102]:
logloss_sticko

0.5379674337922878

In [92]:
X_test['elo'].to_numpy(), y_test.to_numpy()

(array([0.33741685, 0.26124487, 0.33797615, ..., 0.16509364, 0.16257314,
        0.27562489]),
 array([0., 1., 1., ..., 1., 0., 0.]))

In [64]:
y_test.to_numpy()

array([0., 1., 1., ..., 1., 0., 0.])

In [19]:
atp.get_tables()

['categories_atp',
 'categories_mxt',
 'categories_wta',
 'courts',
 'ep_atp',
 'ep_wta',
 'facts_atp',
 'games_atp',
 'games_mxt',
 'games_wta',
 'injury_atp',
 'injury_wta',
 'links',
 'metrics_wta',
 'odds_atp',
 'odds_mxt',
 'odds_wta',
 'players_atp',
 'players_mxt',
 'players_wta',
 'points',
 'ratings_atp',
 'ratings_wta',
 'rounds',
 'seed_atp',
 'seed_mxt',
 'seed_wta',
 'stat_atp',
 'stat_mxt',
 'stat_wta',
 'today_atp',
 'today_mxt',
 'today_wta',
 'tours_atp',
 'tours_mxt',
 'tours_wta',
 'url',
 'version',
 'facts_wta',
 'metrics_atp']

In [7]:
atp.get_table('games_atp')

ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G
i64,i64,i64,i64,str,str
115,105,1,12,"""6-2 6-1""",null
35,223,1,4,"""7-5 6-1""",null
61,110,1,4,"""6-4 6-3""",null
275,181,1,4,"""4-6 6-4 6-4""",null
105,103,1,4,"""6-3 3-6 6-3""",null
…,…,…,…,…,…
96765,56274,20404,4,"""4-6 7-6(10) 7-6(5)""","""02/24/25 00:00:00"""
14861,84377,20404,4,"""6-2 6-3""","""02/24/25 00:00:00"""
90508,58999,20506,3,"""3-6 6-3 6-2""","""02/24/25 00:00:00"""


In [23]:
atp.create_points_games()

P1,P2,Surface,Day,Date
i64,i64,null,null,null
401,921,null,null,null
401,921,null,null,null
401,921,null,null,null
401,921,null,null,null
401,921,null,null,null
…,…,…,…,…
30978,79066,null,null,null
30978,79066,null,null,null
30978,79066,null,null,null


In [24]:
atp.get_table('atp_stat')

Error reading table atp_stat: Command '['mdb-export', '/mnt/c/Program Files (x86)/OnCourt/OnCourt.mdb', 'atp_stat']' returned non-zero exit status 1.


Error: Table atp_stat does not exist in this database.


In [62]:
model_ttt = TrueSkillThroughTime(train_dataset, beta = 0.65, iterations = 5)

In [63]:
test_probs_ttt = model_ttt.fit_dataset(test_dataset, return_pre_match_probs=True)

Iteration =  0 , step =  (14.890971166343009, 5.182221998049987)
Iteration =  1 , step =  (3.736037776706861, 1.4479072145697867)
Iteration =  2 , step =  (1.4806719031861482, 0.509851733378992)
Iteration =  3 , step =  (0.3672129822355088, 0.14592694710301934)
Iteration =  4 , step =  (0.16819834343572992, 0.03712173748539471)
End
Iteration =  0 , step =  (0.34853489546275096, 0.12373774478966576)
Iteration =  1 , step =  (0.0460622205696124, 0.009366288838060566)
Iteration =  2 , step =  (0.02663723880671931, 0.004793480377183412)
Iteration =  3 , step =  (0.017879834348702772, 0.0026477018078892023)
Iteration =  4 , step =  (0.01319536655566056, 0.0019745259252470504)
End
Iteration =  0 , step =  (0.32461039802157954, 0.04522913024160119)
Iteration =  1 , step =  (0.019307278337824307, 0.002066645521956456)
Iteration =  2 , step =  (0.00780031177855689, 0.0014431794789802055)
End
Iteration =  0 , step =  (0.2281184264877738, 0.04827490402571183)
Iteration =  1 , step =  (0.015522832

In [64]:
test_metrics_ttt = binary_metrics_suite(probs=test_probs_ttt, outcomes=test_dataset.outcomes)
print(test_metrics_ttt)

{'accuracy': np.float64(0.7275555555555555), 'accuracy_without_draws': np.float64(0.7275555555555555), 'log_loss': np.float64(0.5353360579616386), 'brier_score': np.float64(0.1804175749281074)}


In [26]:
test_probs_ttt = model_ttt.fit_dataset(test_dataset)

TypeError: TrueSkillThroughTime.add_games() takes from 2 to 3 positional arguments but 5 were given

In [136]:
def objective_function_EloPlus(lambda_reg, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
    
    Parameters:
        gamma: The parameter to optimize
        dataset: Dataset containing competitors
        train_dataset: Training dataset
        test_dataset: Test dataset for evaluation
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = EloPlus(dataset.competitors, lambda_reg=lambda_reg, total_iterations = 50, update_method = 'batched')
    model.fit_dataset(train_dataset)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("lambda_reg of: ", lambda_reg, " gives brier of: ", brier)
    return test_metrics['brier_score']

In [ ]:
def objective_function_Elo(k, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = Elo(dataset.competitors, k=k)
    model.fit_dataset(train_dataset)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("K of: ", k, " gives brier of: ", brier)
    return brier

In [92]:
def objective_function_Glicko(c, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = Glicko(dataset.competitors, c = c)
    model.fit_dataset(train_dataset)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("c of: ", c, " gives brier of: ", brier)
    return brier

In [48]:
def objective_function_Sticko(c, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = Sticko(dataset.competitors, c = 7.301939694943711, h = 260.6, beta = 0.023903153784953272)
    model.fit_dataset(train_dataset)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("c of: ", c, " gives brier of: ", brier)
    return brier

In [69]:
def objective_function_ttt(beta, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = TrueSkillThroughTime(train_dataset, beta = beta)
    model.iterate(5)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True, iterations = 5)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("Beta of: ", beta, " gives brier of: ", brier)
    return brier

In [49]:
result = minimize_scalar(
    objective_function_Sticko,
    args=(dataset, train_dataset, test_dataset),
    method='brent',  # Best general-purpose 1D optimizer
)

c of:  0.0  gives brier of:  0.18709481075415313
c of:  1.0  gives brier of:  0.18689582443859692
c of:  2.6180339999999998  gives brier of:  0.18600621710382773
c of:  5.2360680251559995  gives brier of:  0.18456449175483397
c of:  9.472136091015262  gives brier of:  0.1845489398432723
c of:  7.377102659747564  gives brier of:  0.1841746705159516
c of:  6.559300224511162  gives brier of:  0.18422555629072543
c of:  8.177334199355162  gives brier of:  0.18423592548357012
c of:  7.330955902864865  gives brier of:  0.18417425264080226
c of:  7.305012606453942  gives brier of:  0.18417416283845062
c of:  7.295679585287671  gives brier of:  0.18417418464567598
c of:  7.30745412625843  gives brier of:  0.18417416530210892
c of:  7.304457722875849  gives brier of:  0.1841741624257989
c of:  7.301104772773843  gives brier of:  0.1841741641114022
c of:  7.303569342944174  gives brier of:  0.18417416187823357
c of:  7.302627960934493  gives brier of:  0.18417416153853075
c of:  7.30180656255044

In [80]:
# Get results
print("Optimal variable", result.x, "gives brier of", result.fun)

Optimal variable 9.465461359819516 gives brier of 0.19587134446883256


In [9]:
a = atp.to_ttt_format(sample_games=50000)
h = ttt.History(composition = a[0], times = a[1])

Speedup branch


In [10]:
h.convergence(epsilon = 0.01, iterations = 10)

Iteration =  0 , step =  (14.945056624654391, 5.377917096145063)
Iteration =  1 , step =  (2.812648566510126, 0.9406496200515968)
Iteration =  2 , step =  (0.6736415422089026, 0.19307175884181205)
Iteration =  3 , step =  (0.16691655031202535, 0.050757047021259494)
Iteration =  4 , step =  (0.056196947812871656, 0.016280531610799898)
Iteration =  5 , step =  (0.02045164473815486, 0.005719036192219917)
Iteration =  6 , step =  (0.017492529891368847, 0.0033196865249522922)
Iteration =  7 , step =  (0.016703107565909647, 0.0030154666622732584)
Iteration =  8 , step =  (0.015375995091575945, 0.002739081857586978)
Iteration =  9 , step =  (0.014051774145483975, 0.0025132449104439125)
End


((0.014051774145483975, 0.0025132449104439125), 10)

In [28]:
dataset.matchups

array([[10617, 23969],
       [24225, 33677],
       [13428,  4026],
       ...,
       [34224, 36744],
       [30320, 16045],
       [18871, 34219]], dtype=int32)

P1,P2,Tour,Surface,Day,Date
i64,i64,i64,i8,i64,date
115,105,1,0,2568,1997-01-12
35,223,1,0,2558,1997-01-02
61,110,1,0,2558,1997-01-02
275,181,1,0,2558,1997-01-02
105,103,1,0,2558,1997-01-02
…,…,…,…,…,…
88766,44555,20203,0,12739,2024-11-17
50504,99846,20211,0,12738,2024-11-16
99982,84556,20211,0,12738,2024-11-16


In [114]:
mu = [tp[1].mu for tp in h.learning_curves()[5992]]

In [138]:
players = [5992, 22807, 30470, 19, 47275]

[h.learning_curves()[player][-1] for player in players]

[(12704, N(mu=7.452, sigma=0.439)),
 (12736, N(mu=6.523, sigma=0.400)),
 (12723, N(mu=6.236, sigma=0.375)),
 (11510, N(mu=6.346, sigma=0.489)),
 (12739, N(mu=8.172, sigma=0.434))]

In [120]:
whr = whole_history_rating.Base()
whr.load_games(atp.to_whr_format())

In [135]:
whr.iterate(5)

In [50]:
import time
import numpy as np
from dataclasses import dataclass

# Current approach with Gaussian objects
class Gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    def __mul__(self, other):
        pi1, pi2 = self.sigma**-2, other.sigma**-2
        pi = pi1 + pi2
        tau = (self.mu * pi1 + other.mu * pi2) / pi
        sigma = np.sqrt(1/pi)
        return Gaussian(tau, sigma)

# New approach using just numbers
@dataclass
class FastMessage:
    mu: float
    sigma: float

def multiply_messages(m1: FastMessage, m2: FastMessage) -> FastMessage:
    pi1, pi2 = m1.sigma**-2, m2.sigma**-2
    pi = pi1 + pi2
    tau = (m1.mu * pi1 + m2.mu * pi2) / pi
    sigma = np.sqrt(1/pi)
    return FastMessage(tau, sigma)

def benchmark(n_messages=10000, n_iterations=5):
    """
    Benchmark comparing Gaussian objects vs direct numerical operations.
    
    Args:
        n_messages: Number of message updates to perform
        n_iterations: Number of iterations through the messages
    """
    # Generate random data
    np.random.seed(42)
    means = np.random.randn(n_messages)
    stds = np.abs(np.random.randn(n_messages)) + 1
    
    # Test Gaussian objects
    start = time.time()
    messages = [Gaussian(mu, sigma) for mu, sigma in zip(means, stds)]
    for _ in range(n_iterations):
        for i in range(1, len(messages)):
            messages[i] = messages[i] * messages[i-1]
    gaussian_time = time.time() - start
    
    # Test direct numerical
    start = time.time()
    fast_messages = [FastMessage(mu, sigma) for mu, sigma in zip(means, stds)]
    for _ in range(n_iterations):
        for i in range(1, len(fast_messages)):
            fast_messages[i] = multiply_messages(fast_messages[i], fast_messages[i-1])
    numerical_time = time.time() - start
    
    return {
        'gaussian_time': gaussian_time,
        'numerical_time': numerical_time,
        'speedup': gaussian_time / numerical_time
    }

# Run benchmarks with different sizes
sizes = [1000, 10000, 100000]
for size in sizes:
    print(f"\nBenchmark with {size} messages:")
    results = benchmark(n_messages=size)
    print(f"Gaussian objects: {results['gaussian_time']:.3f} seconds")
    print(f"Direct numerical: {results['numerical_time']:.3f} seconds")
    print(f"Speedup factor: {results['speedup']:.2f}x")


Benchmark with 1000 messages:
Gaussian objects: 0.024 seconds
Direct numerical: 0.008 seconds
Speedup factor: 2.99x

Benchmark with 10000 messages:
Gaussian objects: 0.097 seconds
Direct numerical: 0.152 seconds
Speedup factor: 0.64x

Benchmark with 100000 messages:
Gaussian objects: 0.957 seconds
Direct numerical: 1.899 seconds
Speedup factor: 0.50x
